# Example: Fitting Broad components and Narrow-Line Region Emission Lines 

This notebook steps through using the BEAT routine to fit multiple components to the H$\alpha$ and [N II] $\lambda \lambda$6548, 6583 emission lines in two galaxies: NGC 7319, which only has narrow components, and NGC 3227, which has both broad and narrow components. For fitting the broad components, it is recommended that you follow along with the [corresponding page in the documentation](https://beat-docs.readthedocs.io/en/latest/fitting_broad.html).

In [1]:
import beat
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

# Reading in data

Here we define a function (load_file) to read a spectrum .txt file and create arrays for wavelength, flux, and noise (i.e. error). In this example, the .txt files being read only have 3 columns of data - wavelength, flux, and noise.

We use numpy.loadtxt to read in the data, but one can use any method that returns the required wavelength, flux, and noise arrays. 

In [2]:
def load_file(filepath):
    """reads file and returns wave, flux, and noise arrays to pass to fitting code"""
    #wave, flux, noise = np.loadtxt(filepath, usecols=(0, 1, 2), unpack=True, delimiter='\t', skiprows=1)
    wave, flux, noise = np.loadtxt(filepath, usecols=(0, 1, 2), unpack=True, delimiter=',', skiprows=0)
    #dataset = pd.read_csv(filepath, header= 0, names = ['wave', 'flux', 'noise'],encoding= 'unicode_escape')
    #dataset = pd.read_csv(filepath, header= 0, names = ['wave', 'flux', 'noise'])
    #wave, flux, noise = dataset['wave'], dataset['flux'], dataset['noise']
    
    return wave, flux, noise

# Fitting a NLR spectrum

Run the next two cells without changing anything. This will run BEAT and produce an output folder, in this case called `ngc7319_out`. If BEAT ran succesffully, the `plots` folder within `ngc7319_out` will show three plot files for zero-, one-, and two-component fits.

A description of the parameters in the `main()` function are on the [docs page](https://beat-docs.readthedocs.io/en/latest/index.html).

In [3]:
def main():
    target_param = {'name': 'ngc7319',      # Name of target
                    'red': 0.022,    # Redshift of target
                    'minwidth': 1.5, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 5,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,  # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 2,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6500, 6550), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6750, 6800)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

                        
    fit_instructions = {
                        'line1': # Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6700, #should be redshifted version
                                 'wave_range': 40.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3
                       
    }
    
    fit = beat.Fit(out_dir='', 
                   spec_dir='NLR spectrum',
                   load_file=load_file,
                   target_param=target_param,
                   make_dirs = 'replace', #options: make_new , replace
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   )
    fit.mp_handler()

main()

remaining to fit: 1
Fitting NGC7319.csv
Min Y: 3.48e-16
Max Y: 2.22e-15
components:  0
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    1
 *****************************************************
 ln(ev)=  -369.41635288691003      +/-   5.3071568702583472E-007
 Total Likelihood Evaluations:          200
 Sampling finished. Exiting MultiNest
NGC7319.csv: trying 1 component(s)
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    4
 *****************************************************
components:  1
 ln(ev)=  -113.39692447012020      +/-  0.24332927851958275     
 Total Likelihood Evaluations:         5100
 Sampling finished. Exiting MultiNest
NGC7319.csv fit with 1 components
Average Continuum = 4.94e-16
Standard deviation = 3.2033e-17


# Fitting a BLR spectrum

## Method 1: Fit Everything at Once

In [ ]:
def main():
    target_param = {'name': 'ngc3227',      # Name of target
                    'red': 0.00386,    # Redshift of target
                    'minwidth': 1.5,  # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 50,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 5,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

                          
    fit_instructions = {
                        'line1':  #Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6570, #should be redshifted version
                                 'wave_range': 30.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3
                       
    }
    
    fit = beat.Fit(out_dir='',
                   spec_dir='BLR spectrum',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   make_dirs='make_new',
                   #prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()

## Method 2: Use an Iterative Process

### Step 1: Fitting only the broad lines

In [15]:
def main():
    target_param = {'name': 'ngc3227',      # Name of target
                    'red': 0.00386,    # Redshift of target
                    #'minwidth': 3.83, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'minwidth': 15, #2.16, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 50,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 3,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

                          
    fit_instructions = {
                        'line1':  #Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6570, #should be redshifted version
                                 'wave_range': 30.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3
                       
    }
    
    fit = beat.Fit(out_dir='',
                   spec_dir='BLR spectrum',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   make_dirs = 'make_new',
                   #prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()
main()

remaining to fit: 1
Fitting ngc3227.txt
Min Y: 2.24e-15
Max Y: 1.40e-14
components:  0
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    1
 *****************************************************
 ln(ev)=  -24261.782705180623      +/-   3.1368782375829739E-005
 Total Likelihood Evaluations:          200
 Sampling finished. Exiting MultiNest
ngc3227.txt: trying 1 component(s)
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    4
 *****************************************************

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter      


 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.
 Parameter           10  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.
 Parameter           10  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.
 Parameter           10  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the pri

Process ForkPoolWorker-60:
Process ForkPoolWorker-61:
Process ForkPoolWorker-59:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
 

#### Checking our results:

In [19]:
import pandas as pd
data = pd.read_csv('ngc3227_out_step1/ngc3227_halpha.txt', sep=",", header=0)
data.head()

,index,filename,ncomps,wave_1,width_1,flux_1_A,flux_1_B,wave_2,width_2,flux_2_A,...,flux_1_A_sigma,flux_1_B_sigma,wave_2_sigma,width_2_sigma,flux_2_A_sigma,flux_2_B_sigma,wave_3_sigma,width_3_sigma,flux_3_A_sigma,flux_3_B_sigma
0,0,ngc3227.txt,3,6629.585195,49.954143,2.419954e-14,2.418925e-14,6570.065414,34.127478,9.803505e-14,...,6.504954e-16,5.839262e-16,0.194604,1.221422,5.221025e-15,1.541566e-15,0.362769,0.021469,4.545531e-15,4.827578e-15


### Step 2: Fitting the spectrum with new broad parameters

Now using the new broad components to run a first fit where it's both broad and narrow. Afterwards, going to subtract narrow components to get a deblended spectrum.

In [18]:
def main():
    target_param = {'name': 'ngc3227',      # Name of target
                    'red': 0.00386,    # Redshift of target
                    'minwidth': 1.5, #2.16, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 7,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                   'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 3,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

    prefit_instructions = {'flux': 'unknown',       # Leave as unknown if you want the flux to be able to scale with every new fit
                           'comp1':                         # First broad component
                                   {'name':'broad1',        # Name of broad component
                                    'cen': 6570,       # Centroid of 1st component (taken from output file)
                                    'width': 34.06,       # Width of 1st component (taken from output file)
                                    'flux_free': True},     # Leave as true if you would like the component flux to change
                           'comp2':                 # Second broad component
                                   {'name': 'broad2',       # Name of broad component
                                    'cen': 6629.75,       # Centroid of 2nd component (taken from output file)
                                    'width': 50,       # Width of 2nd component (taken from output file)
                                    'flux_free': False,     # Leave as false if you would like the component to be locked with the first component
                                    'locked_with': 'comp1', # Needed if flux_free=False, component that it is locked with in flux
                                    'flux_ratio': 9.8/2.4}}    # Flux ratio to keep the two components locked in (Flux_1 / Flux_2 = 9.16e-14 / 5.83e-14 = ~1.57)
                   

    fit_instructions = {
                        'line1':  #Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6570, #should be redshifted version
                                 'wave_range': 30.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3
                       
    }
    
    fit = beat.Fit(out_dir='',
                   spec_dir='BLR spectrum',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   make_dirs = 'make_new',
                   save_NLR_removed = True,
                   prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()
main()

remaining to fit: 1
Fitting ngc3227.txt
Min Y: 2.24e-15
Max Y: 1.40e-14
components:  0
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    1
 *****************************************************
 ln(ev)=  -24261.782705180623      +/-   3.1368782375829739E-005
 Total Likelihood Evaluations:          200
 Sampling finished. Exiting MultiNest
ngc3227.txt: trying 1 component(s)
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =   10
 *****************************************************
components:  1
 ln(ev)=  -2175.1205478366237      +/-  0.32578100363824336     
 Total Likelihood Evaluations:        24088
 Sampling finished. Exiting MultiNest
ngc3227.txt: trying 2 component(s)
 *****************************************************
 MultiNest 

ERROR: Interrupt received: Terminating
Process ForkPoolWorker-66:
Process ForkPoolWorker-67:
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/Users/juliafalcone/opt/anaconda3/lib/py

### Step 3: Fitting the NLR-subtracted spectrum

Now fitting first deblended spectrum

In [10]:
def main():
    target_param = {'name': 'ngc3227',      # Name of target
                    'red': 0.00386,    # Redshift of target
                    'minwidth': 10, #2.16, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 50,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 3,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]
                  

    fit_instructions = {
                        'line1':  #Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6570, #should be redshifted version
                                 'wave_range': 30.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3
                       
    }
    
    fit = beat.Fit(out_dir='',
                   spec_dir='ngc3227_out_2/NLR_removed',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   make_dirs = 'make_new',
                   #prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()
main()

remaining to fit: 1
Fitting ngc3227_halpha_NLRremoved.txt
Min Y: 2.24e-15
Max Y: 6.91e-15
components:  0
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    1
 *****************************************************
 ln(ev)=  -6502.4746324164844      +/-                       NaN
 Total Likelihood Evaluations:          200
 Sampling finished. Exiting MultiNest
ngc3227_halpha_NLRremoved.txt: trying 1 component(s)
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    4
 *****************************************************

 MultiNest Warning!
 Parameter            1  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            1  of mode            1  is converging towards the edge of the prior.




 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            5  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            5  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            3  of mode            1  is converging towards the edge of the prior.
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter       


 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.
 Parameter            9  of mode            1  is converging towards the edge of the prior.
 Parameter           12  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.
 Parameter            9  of mode            1  is converging towards the edge of the prior.
 Parameter           12  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter            4  of mode            1  is converging towards the edge of the prior.
 Parameter            8  of mode            1  is converging towards the edge of the prior.
 Parameter       

Process ForkPoolWorker-17:
Process ForkPoolWorker-16:
Process ForkPoolWorker-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
 

### Step 4: Fitting the spectrum with newer broad parameters

Now taking the broad fit from the deblended spectrum and putting that back in the main spectrum to fit it a second time, and refitting with narrow components. 

In [13]:
def main():
    target_param = {'name': 'ngc3227',      # Name of target
                    'red': 0.00386,    # Redshift of target
                    'minwidth': 1.5, #2.16, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 5,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'start': 3180,       # Minimum array value of used spectrum [array units]
                    #'end': 1100,        # STIS Maximum array value of used spectrum [array units]
                    'end': 3910,        # APO Maximum array value of used spectrum [array units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 3,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

    prefit_instructions = {'flux': 'unknown',       # Leave as unknown if you want the flux to be able to scale with every new fit
                           'comp1':                         # First broad component
                                   {'name':'broad1',        # Name of broad component
                                    'cen': 6588,       # Centroid of 1st component (taken from output file)
                                    'width': 48.3,       # Width of 1st component (taken from output file)
                                    'flux_free': True},     # Leave as true if you would like the component flux to change
                           'comp2':                 # Second broad component
                                   {'name': 'broad2',       # Name of broad component
                                    'cen': 6570.,       # Centroid of 2nd component (taken from output file)
                                    'width': 21.2,       # Width of 2nd component (taken from output file)
                                    'flux_free': False,     # Leave as false if you would like the component to be locked with the first component
                                    'locked_with': 'comp1', # Needed if flux_free=False, component that it is locked with in flux
                                    'flux_ratio': 13.4/7.45}}    # Flux ratio to keep the two components locked in (Flux_1 / Flux_2 = 9.16e-14 / 5.83e-14 = ~1.57)
                    

    fit_instructions = {
                        'line1':  #Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6570, #should be redshifted version
                                 'wave_range': 30.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3               
    }
    
    fit = beat.Fit(out_dir='',
                   spec_dir='BLR spectrum',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   make_dirs = 'make_new',
                   prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()
main()

remaining to fit: 1
Fitting ngc3227.txt
Min Y: 2.24e-15
Max Y: 1.40e-14
components:  0
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    1
 *****************************************************
 ln(ev)=  -24261.782705180623      +/-   3.1368782375829739E-005
 Total Likelihood Evaluations:          200
 Sampling finished. Exiting MultiNest
ngc3227.txt: trying 1 component(s)
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =   10
 *****************************************************
components:  1
ngc3227.txt: trying 2 component(s)
 ln(ev)=  -2126.4285569785325      +/-  0.32604308688533729     
 Total Likelihood Evaluations:        36570
 Sampling finished. Exiting MultiNest
 *****************************************************
 MultiNest 

ERROR: Interrupt received: Terminating
Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in 

  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/Users/juliafalcone/opt/anaconda3/lib/python3.7/multiprocessing/connect

### (Optional) Step 5: Repeating Step 3

Now refitting the broad region for the second time, using the new deblended spectrum.

In [34]:
def main():
    target_param = {'name': 'ngc3227',      # Name of target
                    'red': 0.00386,    # Redshift of target
                    'minwidth': 15, #2.16, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 50,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'start': 0, #3180,       # Minimum array value of used spectrum [array units]
                    #'end': 1100,        # STIS Maximum array value of used spectrum [array units]
                    'end': 730, #3910,        # APO Maximum array value of used spectrum [array units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 3,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

    prefit_instructions = {'flux': 'unknown',       # Leave as unknown if you want the flux to be able to scale with every new fit
                           'comp1':                         # First broad component
                                   {'name':'broad1',        # Name of broad component
                                    'cen': 6570,       # Centroid of 1st component (taken from output file)
                                    'width': 34.06,       # Width of 1st component (taken from output file)
                                    'flux_free': True},     # Leave as true if you would like the component flux to change
                           'comp2':                 # Second broad component
                                   {'name': 'broad2',       # Name of broad component
                                    'cen': 6629.75,       # Centroid of 2nd component (taken from output file)
                                    'width': 50,       # Width of 2nd component (taken from output file)
                                    'flux_free': False,     # Leave as false if you would like the component to be locked with the first component
                                    'locked_with': 'comp1', # Needed if flux_free=False, component that it is locked with in flux
                                    'flux_ratio': 9.8/2.4}}    # Flux ratio to keep the two components locked in (Flux_1 / Flux_2 = 9.16e-14 / 5.83e-14 = ~1.57)
    

    fit_instructions = {
                        'line1':  #Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6570, #should be redshifted version
                                 'wave_range': 30.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3                 
    }
    
    fit = beat.Fit(out_dir='',
                   spec_dir='ngc3227_out_step4/BLR_removed',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   make_dirs = 'make_new',
                   fit_instructions=fit_instructions,
                   #prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()
main()

### (Optional) Step 6: Repeating Step 4

Now fitting the main spectrum for the third time

In [36]:
def main():
    target_param = {'name': 'ngc3227',      # Name of target
                    'red': 0.00386,    # Redshift of target
                    'minwidth': 1.5, #2.16, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 5,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'start': 3180,       # Minimum array value of used spectrum [array units]
                    #'end': 1100,        # STIS Maximum array value of used spectrum [array units]
                    'end': 3910,        # APO Maximum array value of used spectrum [array units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 6400,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 6900,#6725,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 3,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 3}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (6400, 6450), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                        'continuum2': (6800, 6850)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]

    prefit_instructions = {'flux': 'unknown',       # Leave as unknown if you want the flux to be able to scale with every new fit
                           'comp1':                         # First broad component
                                   {'name':'broad1',        # Name of broad component
                                    'cen': 6584,       # Centroid of 1st component (taken from output file)
                                    'width': 49.71,       # Width of 1st component (taken from output file)
                                    'flux_free': True},     # Leave as true if you would like the component flux to change
                           'comp2':                 # Second broad component
                                   {'name': 'broad2',       # Name of broad component
                                    'cen': 6570.,       # Centroid of 2nd component (taken from output file)
                                    'width': 21.6,       # Width of 2nd component (taken from output file)
                                    'flux_free': False,     # Leave as false if you would like the component to be locked with the first component
                                    'locked_with': 'comp1', # Needed if flux_free=False, component that it is locked with in flux
                                    'flux_ratio': 10.5/3.83}}    # Flux ratio to keep the two components locked in (Flux_1 / Flux_2 = 9.16e-14 / 5.83e-14 = ~1.57)
                   

    fit_instructions = {
                        'line1':  #Instructions for narrow line component H-alpha (6563)
                                {'name': 'halpha', 
                                 'wave': 6562.8, 
                                 'minwave': 6570, #should be redshifted version
                                 'wave_range': 30.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                         'line2': # Instructions for narrow line component [N II] (6583)
                                {'name': 'nii', 
                                 'wave': 6583.46, 
                                 'flux_free': True},       # Allow flux to vary freely
                        'line3': # Instructions for narrow line component [N II] (6548)
                                {'name': 'nii', 
                                  'wave': 6548.04, 
                                  'flux_free': False,     # Lock flux with another component
                                  'locked_with': 'line2', # Lock flux w/ 'line2' ([N II] 6583)
                                  'flux_ratio': 3},       # Flux ratio between line2/line3 = 3                                   
    }
    
    fit = beat.Fit(out_dir='',
                   spec_dir='BLR spectrum',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   make_dirs = 'make_new',
                   prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()
main()